# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools FFTW SpecialFunctions"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=1
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

Installing Julia 1.6.2 on the current Colab Runtime...
2021-07-26 09:00:51 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.2-linux-x86_64.tar.gz [112946671/112946671] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package FFTW...
Installing Julia package SpecialFunctions...
Installing Julia package CUDA...


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.6.2
Commit 1b93d53fc4 (2021-07-14 15:36 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)
Environment:
  JULIA_NUM_THREADS = 1


In [2]:
using CUDA
CUDA.versioninfo()

CUDA toolkit 11.3.1, artifact installation
CUDA driver 11.2.0
NVIDIA driver 460.32.3

Libraries: 
- CUBLAS: 11.5.1
- CURAND: 10.2.4
- CUFFT: 10.4.2
- CUSOLVER: 11.1.2
- CUSPARSE: 11.6.0
- CUPTI: 14.0.0
- NVML: 11.0.0+460.32.3


- CUDNN: 8.20.0 (for CUDA 11.3.0)


- CUTENSOR: 1.3.0 (for CUDA 11.2.0)

Toolchain:
- Julia: 1.6.2
- LLVM: 11.0.1
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4, 6.5, 7.0
- Device capability support: sm_35, sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75, sm_80

1 device:
  0: Tesla K80 (sm_37, 11.170 GiB / 11.173 GiB available)


In [3]:
using BenchmarkTools

M = rand(2048, 2048)
@benchmark M^2

BenchmarkTools.Trial: 10 samples with 1 evaluation.
 Range (min … max):  497.959 ms … 531.661 ms  ┊ GC (min … max): 0.00% … 0.57%
 Time  (median):     512.504 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   510.354 ms ±  11.093 ms  ┊ GC (mean ± σ):  0.24% ± 0.33%

  ▁▁▁  ▁              ▁          ▁▁ █                         ▁  
  ███▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁██▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  498 ms           Histogram: frequency by time          532 ms <

 Memory estimate: 32.00 MiB, allocs estimate: 2.

In [ ]:
if ENV["COLAB_GPU"] == "1"
    using CUDA

    M_gpu = cu(M)
    @benchmark CUDA.@sync M_gpu^2
else
    println("No GPU found.")
end

BenchmarkTools.Trial: 1248 samples with 1 evaluation.
 Range (min … max):  2.498 ms …  11.012 ms  ┊ GC (min … max): 0.00% … 48.71%
 Time  (median):     3.915 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.001 ms ± 358.268 μs  ┊ GC (mean ± σ):  0.11% ±  1.38%

                              ▂██▅▅▄▃▅▅▃▄▁     ▁               
  ▇▄▄▄▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▄▆▁▁▁▆▆▇████████████▅▇▆█▆█▁▆▆▅▁▁▁▁▁▁▁▁▅ █
  2.5 ms       Histogram: log(frequency) by time      5.19 ms <

 Memory estimate: 33.12 KiB, allocs estimate: 2112.

In [4]:
using Pkg

In [5]:
Pkg.add(PackageSpec(url="https://github.com/f-fathurrahman/PWDFT.jl"))

     Cloning git-repo `https://github.com/f-fathurrahman/PWDFT.jl`
    Updating git-repo `https://github.com/f-fathurrahman/PWDFT.jl`
    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
   Installed Libxc ────────── v0.3.6
   Installed BinaryProvider ─ v0.5.10
   Installed LibSymspg ────── v0.4.0
   Installed Libxc_jll ────── v5.1.5+2
    Updating `~/.julia/environments/v1.6/Project.toml`
  [f0909338] + PWDFT v0.2.0 `https://github.com/f-fathurrahman/PWDFT.jl#master`
    Updating `~/.julia/environments/v1.6/Manifest.toml`
  [b99e7846] + BinaryProvider v0.5.10
  [2098c8a6] + LibSymspg v0.4.0
  [66e17ffc] + Libxc v0.3.6
  [f0909338] + PWDFT v0.2.0 `https://github.com/f-fathurrahman/PWDFT.jl#master`
  [a56a6d9d] + Libxc_jll v5.1.5+2
    Building LibSymspg → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/29f99e23b35b74521cae927d8ab9af5c7ee2dd61/build.log`
Precompiling project...
  ✓ LibSymspg
  ✓ Libxc_jll
  ✓ Libxc
  ✓ PWDFT
  4 dependenci

In [6]:
Pkg.add(PackageSpec(url="https://github.com/f-fathurrahman/PWDFTCUDA.jl"))

     Cloning git-repo `https://github.com/f-fathurrahman/PWDFTCUDA.jl`
    Updating git-repo `https://github.com/f-fathurrahman/PWDFTCUDA.jl`
   Resolving package versions...
    Updating `~/.julia/environments/v1.6/Project.toml`
  [f2219c79] + PWDFTCUDA v0.1.0 `https://github.com/f-fathurrahman/PWDFTCUDA.jl#master`
    Updating `~/.julia/environments/v1.6/Manifest.toml`
  [f2219c79] + PWDFTCUDA v0.1.0 `https://github.com/f-fathurrahman/PWDFTCUDA.jl#master`
Precompiling project...
  ✓ PWDFTCUDA
  1 dependency successfully precompiled in 12 seconds (49 already precompiled)


In [7]:
using PWDFTCUDA

┌ Info: Precompiling PWDFTCUDA [f2219c79-a41e-4b93-8274-7973f04b47ed]
└ @ Base loading.jl:1342
┌ Warning: Package PWDFTCUDA does not have Printf in its dependencies:
│ - If you have PWDFTCUDA checked out for development and have
│   added Printf as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with PWDFTCUDA
└ Loading Printf into PWDFTCUDA from project dependency, future warnings for PWDFTCUDA are suppressed.


In [8]:
using Printf
using Random

using CUDA
using PWDFT
using PWDFTCUDA

const DIR_PWDFT = joinpath(dirname(pathof(PWDFT)),"..")
const DIR_PSP = joinpath(DIR_PWDFT, "pseudopotentials", "pade_gth")
const DIR_STRUCTURES = joinpath(DIR_PWDFT, "structures")

function get_default_psp( atoms::Atoms; xcfunc="VWN" )

    DIR_PWDFT = joinpath( dirname(pathof(PWDFT)), "..")
    if xcfunc == "VWN"
        DIR_PSP = joinpath(DIR_PWDFT, "pseudopotentials", "pade_gth")
        ALL_PSP = PWDFT.ALL_PADE_PSP
    elseif xcfunc == "PBE"
        DIR_PSP = joinpath(DIR_PWDFT, "pseudopotentials", "pbe_gth")
        ALL_PSP = PWDFT.ALL_PBE_PSP
    else
        errmsg = @sprintf("xcfunc in get_default_psp is not known %s\n", xcfunc)
        error(errmsg)
    end

    Nspecies = atoms.Nspecies
    pspfiles = Array{String}(undef,Nspecies)

    SpeciesSymbols = atoms.SpeciesSymbols
    for isp = 1:Nspecies
        atsymb = SpeciesSymbols[isp]
        pspfiles[isp] = joinpath(DIR_PSP, ALL_PSP[atsymb][1])
    end
    return pspfiles
end

function main_CPU(molname)

    Random.seed!(1234)

    filename = joinpath(DIR_STRUCTURES, "DATA_G2_mols", molname*".xyz")
    atoms = Atoms(ext_xyz_file=filename)
    pspfiles = get_default_psp(atoms)
    ecutwfc = 15.0

    Nspin = 1
    
    Ham_cpu = Hamiltonian( atoms, pspfiles, ecutwfc, use_symmetry=false, Nspin=Nspin )
    println( Ham_cpu )

    KS_solve_Emin_PCG!( Ham_cpu, skip_initial_diag=true, startingrhoe=:random )
    
    @time KS_solve_Emin_PCG!( Ham_cpu, skip_initial_diag=true, startingrhoe=:random )

end

function main_GPU(molname)

    Random.seed!(1234)

    filename = joinpath(DIR_STRUCTURES, "DATA_G2_mols", molname*".xyz")
    atoms = Atoms(ext_xyz_file=filename)
    pspfiles = get_default_psp(atoms)

    ecutwfc = 15.0

    Nspin = 1

    Ham = CuHamiltonian( atoms, pspfiles, ecutwfc, use_symmetry=false, Nspin=Nspin )
    KS_solve_Emin_PCG!( Ham, skip_initial_diag=true, startingrhoe=:random )
    
    @time KS_solve_Emin_PCG!( Ham, skip_initial_diag=true, startingrhoe=:random )

end

main_GPU (generic function with 1 method)

In [ ]:
main_CPU("H2O")


                                  -----------
                                  Hamiltonian
                                  -----------

size (MiB) =           25.08247

xcfunc     = VWN
xc_calc    = LibxcXCCalculator([0.0], [0.0])


                                     -----
                                     Atoms
                                     -----

Natoms   =     3
Nspecies =     2

Species   1: O
Species   2: H

Cartesian coordinates in bohr:

    O       9.4486306273      10.8909433056      10.5754913234
    H       9.4486306273      12.3332559838       9.4486306273
    H       9.4486306273       9.4486306273       9.4486306273

Direct lattice vectors: (v1, v2, and v3 are given by column)

     18.8972612546       0.0000000000       0.0000000000
      0.0000000000      21.7818866111       0.0000000000
      0.0000000000       0.0000000000      20.0241219507

Fractional coordinates:

    O       0.5000000000       0.5000000000       0.5281375807
    H       0.500000000

In [9]:
main_CPU("H2O")


                                  -----------
                                  Hamiltonian
                                  -----------

size (MiB) =           25.08247

xcfunc     = VWN
xc_calc    = LibxcXCCalculator([0.0], [0.0])


                                     -----
                                     Atoms
                                     -----

Natoms   =     3
Nspecies =     2

Species   1: O
Species   2: H

Cartesian coordinates in bohr:

    O       9.4486306273      10.8909433056      10.5754913234
    H       9.4486306273      12.3332559838       9.4486306273
    H       9.4486306273       9.4486306273       9.4486306273

Direct lattice vectors: (v1, v2, and v3 are given by column)

     18.8972612546       0.0000000000       0.0000000000
      0.0000000000      21.7818866111       0.0000000000
      0.0000000000       0.0000000000      20.0241219507

Fractional coordinates:

    O       0.5000000000       0.5000000000       0.5281375807
    H       0.500000000

In [10]:
main_GPU("H2O")


Minimizing Kohn-Sham energy using PCG (CUDA version)
----------------------------------------------------
NiterMax  = 200
α_t       = 3.000000e-05
conv_thr  = 1.000000e-06
Using Polak-Ribiere formula for CG_BETA

Effective GPU memory usage: 4.75% (543.375 MiB/11.173 GiB)
CUDA allocator usage: 361.520 MiB
Memory pool usage: 361.520 MiB (137.269 MiB allocated, 224.251 MiB cached)

Emin_PCG step        1 =      52.3791684486 2.3066766e+01
Emin_PCG step        2 =       8.0923112117 4.4286857e+01
Emin_PCG step        3 =      -2.0436991270 1.0136010e+01
Emin_PCG step        4 =      -6.6818925877 4.6381935e+00
Emin_PCG step        5 =     -10.3873590404 3.7054665e+00
Emin_PCG step        6 =     -12.5802120561 2.1928530e+00
Emin_PCG step        7 =     -14.2058029973 1.6255909e+00
Emin_PCG step        8 =     -14.8386087798 6.3280578e-01
Emin_PCG step        9 =     -15.3224045855 4.8379581e-01
Emin_PCG step       10 =     -15.5309730873 2.0856850e-01
Emin_PCG step       11 =     -15.7646

In [ ]:
main_GPU("H2O")


Minimizing Kohn-Sham energy using PCG (CUDA version)
----------------------------------------------------
NiterMax  = 200
α_t       = 3.000000e-05
conv_thr  = 1.000000e-06
Using Polak-Ribiere formula for CG_BETA

Effective GPU memory usage: 12.96% (1.912 GiB/14.756 GiB)
CUDA allocator usage: 127.549 MiB
Memory pool usage: 127.549 MiB (127.549 MiB allocated, 0 bytes cached)

Emin_PCG step        1 =      52.7133847048 2.3027387e+01
Emin_PCG step        2 =       7.9540504634 4.4759334e+01
Emin_PCG step        3 =      -2.7044593883 1.0658510e+01
Emin_PCG step        4 =      -7.2286824093 4.5242230e+00
Emin_PCG step        5 =     -10.8005043918 3.5718220e+00
Emin_PCG step        6 =     -13.2313411620 2.4308368e+00
Emin_PCG step        7 =     -14.7217136268 1.4903725e+00
Emin_PCG step        8 =     -15.6614973040 9.3978368e-01
Emin_PCG step        9 =     -16.0068824957 3.4538519e-01
Emin_PCG step       10 =     -16.1265732768 1.1969078e-01
Emin_PCG step       11 =     -16.145524894

In [ ]:
main_GPU("CO2")


Minimizing Kohn-Sham energy using PCG (CUDA version)
----------------------------------------------------
NiterMax  = 200
α_t       = 3.000000e-05
conv_thr  = 1.000000e-06
Using Polak-Ribiere formula for CG_BETA

Effective GPU memory usage: 12.96% (1.912 GiB/14.756 GiB)
CUDA allocator usage: 990.588 MiB
Memory pool usage: 990.588 MiB (990.588 MiB allocated, 0 bytes cached)

Emin_PCG step        1 =     109.1266166107 4.7716837e+01
Emin_PCG step        2 =      19.5945240126 8.9532093e+01
Emin_PCG step        3 =      -4.9218551233 2.4516379e+01
Emin_PCG step        4 =     -18.9181528961 1.3996298e+01
Emin_PCG step        5 =     -26.2697661702 7.3516133e+00
Emin_PCG step        6 =     -30.2653464820 3.9955803e+00
Emin_PCG step        7 =     -32.5488445563 2.2834981e+00
Emin_PCG step        8 =     -34.0343762169 1.4855317e+00
Emin_PCG step        9 =     -34.5859314681 5.5155525e-01
Emin_PCG step       10 =     -34.8122961813 2.2636471e-01
Emin_PCG step       11 =     -34.977903410

In [ ]:
main_CPU("CO2")


                                  -----------
                                  Hamiltonian
                                  -----------

size (MiB) =           27.76590

xcfunc     = VWN
xc_calc    = LibxcXCCalculator([0.0], [0.0])


                                     -----
                                     Atoms
                                     -----

Natoms   =     3
Nspecies =     2

Species   1: C
Species   2: O

Cartesian coordinates in bohr:

    C       9.4486306273       9.4486306273      11.6759714429
    O       9.4486306273       9.4486306273      13.9033122584
    O       9.4486306273       9.4486306273       9.4486306273

Direct lattice vectors: (v1, v2, and v3 are given by column)

     18.8972612546       0.0000000000       0.0000000000
      0.0000000000      18.8972612546       0.0000000000
      0.0000000000       0.0000000000      23.3519428857

Fractional coordinates:

    C       0.5000000000       0.5000000000       0.5000000000
    O       0.500000000

In [ ]:
main_CPU("N2")


                                  -----------
                                  Hamiltonian
                                  -----------

size (MiB) =           23.49359

xcfunc     = VWN
xc_calc    = LibxcXCCalculator([0.0], [0.0])


                                     -----
                                     Atoms
                                     -----

Natoms   =     2
Nspecies =     1

Species   1: N

Cartesian coordinates in bohr:

    N       9.4486306273       9.4486306273      11.5839833545
    N       9.4486306273       9.4486306273       9.4486306273

Direct lattice vectors: (v1, v2, and v3 are given by column)

     18.8972612546       0.0000000000       0.0000000000
      0.0000000000      18.8972612546       0.0000000000
      0.0000000000       0.0000000000      21.0326139818

Fractional coordinates:

    N       0.5000000000       0.5000000000       0.5507628945
    N       0.5000000000       0.5000000000       0.4492371055

                                     

In [ ]:
main_GPU("N2")


Minimizing Kohn-Sham energy using PCG (CUDA version)
----------------------------------------------------
NiterMax  = 200
α_t       = 3.000000e-05
conv_thr  = 1.000000e-06
Using Polak-Ribiere formula for CG_BETA

Effective GPU memory usage: 12.96% (1.912 GiB/14.756 GiB)
CUDA allocator usage: 138.563 MiB
Memory pool usage: 138.563 MiB (138.563 MiB allocated, 0 bytes cached)

Emin_PCG step        1 =      66.2310938957 2.9221083e+01
Emin_PCG step        2 =      11.0561523346 5.5174942e+01
Emin_PCG step        3 =      -2.0430908616 1.3099243e+01
Emin_PCG step        4 =      -9.2066440157 7.1635532e+00
Emin_PCG step        5 =     -13.9926704834 4.7860265e+00
Emin_PCG step        6 =     -16.3040493901 2.3113789e+00
Emin_PCG step        7 =     -17.8248892005 1.5208398e+00
Emin_PCG step        8 =     -18.5465453326 7.2165613e-01
Emin_PCG step        9 =     -18.9763602870 4.2981495e-01
Emin_PCG step       10 =     -19.0422174358 6.5857149e-02
Emin_PCG step       11 =     -19.069056409

In [ ]:
main_GPU("C2H6")


Minimizing Kohn-Sham energy using PCG (CUDA version)
----------------------------------------------------
NiterMax  = 200
α_t       = 3.000000e-05
conv_thr  = 1.000000e-06
Using Polak-Ribiere formula for CG_BETA

Effective GPU memory usage: 12.96% (1.912 GiB/14.756 GiB)
CUDA allocator usage: 1011.437 MiB
Memory pool usage: 1011.437 MiB (1011.437 MiB allocated, 0 bytes cached)

Emin_PCG step        1 =     100.1871858667 4.1223851e+01
Emin_PCG step        2 =      22.2311309783 7.7956055e+01
Emin_PCG step        3 =       3.1467170053 1.9084414e+01
Emin_PCG step        4 =      -6.3212541944 9.4679712e+00
Emin_PCG step        5 =     -10.6639315943 4.3426774e+00
Emin_PCG step        6 =     -12.8484768745 2.1845453e+00
Emin_PCG step        7 =     -14.0476826729 1.1992058e+00
Emin_PCG step        8 =     -14.4847390075 4.3705633e-01
Emin_PCG step        9 =     -14.5855920858 1.0085308e-01
Emin_PCG step       10 =     -14.6119404203 2.6348334e-02
Emin_PCG step       11 =     -14.616166

In [ ]:
main_CPU("C2H6")


                                  -----------
                                  Hamiltonian
                                  -----------

size (MiB) =           35.97736

xcfunc     = VWN
xc_calc    = LibxcXCCalculator([0.0], [0.0])


                                     -----
                                     Atoms
                                     -----

Natoms   =     8
Nspecies =     2

Species   1: C
Species   2: H

Cartesian coordinates in bohr:

    C      11.1162062186      11.3741802909      13.0758427621
    C      11.1162062186      11.3741802909      10.1951102414
    H      11.1162062186      13.2997299545      13.8223223762
    H       9.4486306273      10.4114045142      13.8223223762
    H      12.7837818099      10.4114045142      13.8223223762
    H      11.1162062186       9.4486306273       9.4486306273
    H       9.4486306273      12.3369560676       9.4486306273
    H      12.7837818099      12.3369560676       9.4486306273

Direct lattice vectors: (v1, v

In [ ]:
main_CPU("HF")


                                  -----------
                                  Hamiltonian
                                  -----------

size (MiB) =           23.02059

xcfunc     = VWN
xc_calc    = LibxcXCCalculator([0.0], [0.0])


                                     -----
                                     Atoms
                                     -----

Natoms   =     2
Nspecies =     2

Species   1: F
Species   2: H

Cartesian coordinates in bohr:

    F       9.4486306273       9.4486306273      11.2134288483
    H       9.4486306273       9.4486306273       9.4486306273

Direct lattice vectors: (v1, v2, and v3 are given by column)

     18.8972612546       0.0000000000       0.0000000000
      0.0000000000      18.8972612546       0.0000000000
      0.0000000000       0.0000000000      20.6620594756

Fractional coordinates:

    F       0.5000000000       0.5000000000       0.5427062516
    H       0.5000000000       0.5000000000       0.4572937484

                      

In [ ]:
main_GPU("HF")


Minimizing Kohn-Sham energy using PCG (CUDA version)
----------------------------------------------------
NiterMax  = 200
α_t       = 3.000000e-05
conv_thr  = 1.000000e-06
Using Polak-Ribiere formula for CG_BETA

Effective GPU memory usage: 12.96% (1.912 GiB/14.756 GiB)
CUDA allocator usage: 412.856 MiB
Memory pool usage: 412.856 MiB (412.856 MiB allocated, 0 bytes cached)

Emin_PCG step        1 =      49.5613171770 2.2972254e+01
Emin_PCG step        2 =       4.8236867370 4.4737630e+01
Emin_PCG step        3 =      -5.7618018573 1.0585489e+01
Emin_PCG step        4 =     -11.7601016930 5.9982998e+00
Emin_PCG step        5 =     -15.3949187355 3.6348170e+00
Emin_PCG step        6 =     -17.6956031790 2.3006844e+00
Emin_PCG step        7 =     -19.5562170484 1.8606139e+00
Emin_PCG step        8 =     -20.7578737933 1.2016567e+00
Emin_PCG step        9 =     -21.7359881476 9.7811435e-01
Emin_PCG step       10 =     -22.1637344318 4.2774628e-01
Emin_PCG step       11 =     -22.463125407

# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />